<a href="https://colab.research.google.com/github/remotesensinginfo/pb_gee_tools/blob/main/examples/applications/burnt_vegetation/03_Burnt_Mapping_LA_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# The Google Earth Engine module
import ee

# The datetime module is used to specify the dates
# to search for imagery
import datetime

# Import the geemap (https://geemap.org/) module which
# has a visualisation tool
import geemap

# Geopandas allows us to read the shapefile used to
# define the region of interest (ROI)
import geopandas

# The colab module to access data from your google drive
from google.colab import drive

In [2]:
try:
  import pb_gee_tools
  import pb_gee_tools.datasets
except:
  !git clone https://github.com/remotesensinginfo/pb_gee_tools.git
  !pip install ./pb_gee_tools/.
  import pb_gee_tools
  import pb_gee_tools.datasets

In [3]:
ee_prj_name = "ee-pb-dev"  # <==== Replace this with your own EE project string
ee.Authenticate()
ee.Initialize(project=ee_prj_name)

In [4]:
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# The region of interest
vec_roi_file = "/content/drive/MyDrive/burnt_veg/la_2025_roi_poly.geojson"

# Dates before the fire
pre_burn_start_date = datetime.datetime(year=2024, month=9, day=1)
pre_burn_end_date = datetime.datetime(year=2024, month=11, day=30)

# Dates after the fire
post_burn_start_date = datetime.datetime(year=2025, month=1, day=8)
post_burn_end_date = datetime.datetime(year=2025, month=1, day=20)

# No Data Vaue
out_no_data_val = 0.0

In [6]:
# Read the vector layer and make sure it is project using WGS84 (EPSG:4326)
vec_gdf = geopandas.read_file(vec_roi_file).to_crs(4326)

# Get layer bbox: minx, miny, maxx, maxy
gp_bbox = vec_gdf.total_bounds

# Create the GEE geometry from the bbox.
roi_west = gp_bbox[0]
roi_east = gp_bbox[2]
roi_north = gp_bbox[3]
roi_south = gp_bbox[1]
tile_aoi = ee.Geometry.BBox(roi_west, roi_south, roi_east, roi_north)

In [7]:
# Get the landsat image collection
pre_burn_s2_img_col = pb_gee_tools.datasets.get_sen2_sr_collection(
    aoi=tile_aoi,
    start_date=pre_burn_start_date,
    end_date=pre_burn_end_date,
    cloud_thres=70,
)

# Create a median composite
pre_burn_s2_median = pre_burn_s2_img_col.median().unmask(out_no_data_val).clip(tile_aoi)

In [8]:
# Get the landsat image collection
post_burn_s2_img_col = pb_gee_tools.datasets.get_sen2_sr_collection(
    aoi=tile_aoi,
    start_date=post_burn_start_date,
    end_date=post_burn_end_date,
    cloud_thres=70,
)

# Create a median composite
post_burn_s2_median = (
    post_burn_s2_img_col.median().unmask(out_no_data_val).clip(tile_aoi)
)

In [9]:
post_burn_s2_median

In [10]:
map_obj = geemap.Map()

# Create the centre point for the map
cnt_pt_x = roi_west + (roi_east - roi_west) / 2
cnt_pt_y = roi_south + (roi_north - roi_south) / 2
cnt_pt = ee.Geometry.Point([cnt_pt_x, cnt_pt_y])
# Specify the centre point and scale of the map
map_obj.centerObject(cnt_pt, 12)

# Add Google Satellite Imagery Basemap
gg_img_url = "http://www.google.cn/maps/vt?lyrs=s@189&gl=cn&x={x}&y={y}&z={z}"
map_obj.add_tile_layer(gg_img_url, name="Google Imagery", attribution="Google")

# Specify the visualisation options.
vis_params = {
    "min": [1320, 810, 20],
    "max": [4050, 4070, 1620],
    "bands": ["B8", "B11", "B4"],
}
map_obj.addLayer(pre_burn_s2_median, vis_params, "Pre-Burnt Sentinel-2 Median")
map_obj.addLayer(post_burn_s2_median, vis_params, "Post-Burnt Sentinel-2 Median")

map_obj

Map(center=[34.083568028567605, -118.61450081926833], controls=(WidgetControl(options=['position', 'transparen…

In [11]:
# Calculate the Normalised Burn Ratio (NBR) pre and post fire using the
# median composite
pre_med_nbr = pre_burn_s2_median.expression(
    "(NIR-SWIR)/(NIR+SWIR)",
    {
        "NIR": pre_burn_s2_median.select("B8"),
        "SWIR": pre_burn_s2_median.select("B11"),
    },
).rename("NBR")

post_med_nbr = post_burn_s2_median.expression(
    "(NIR-SWIR)/(NIR+SWIR)",
    {
        "NIR": post_burn_s2_median.select("B8"),
        "SWIR": post_burn_s2_median.select("B11"),
    },
).rename("NBR")


In [13]:
map_obj = geemap.Map()

# Create the centre point for the map
cnt_pt_x = roi_west + (roi_east - roi_west) / 2
cnt_pt_y = roi_south + (roi_north - roi_south) / 2
cnt_pt = ee.Geometry.Point([cnt_pt_x, cnt_pt_y])
# Specify the centre point and scale of the map
map_obj.centerObject(cnt_pt, 12)

# Add Google Satellite Imagery Basemap
gg_img_url = "http://www.google.cn/maps/vt?lyrs=s@189&gl=cn&x={x}&y={y}&z={z}"
map_obj.add_tile_layer(gg_img_url, name="Google Imagery", attribution="Google")

# Specify the visualisation options.
vis_params = {"min": [-0.2], "max": [0.2], "bands": ["NBR"]}
map_obj.addLayer(pre_med_nbr, vis_params, "Pre-Burnt NBR")
map_obj.addLayer(post_med_nbr, vis_params, "Post-Burnt NBR")

map_obj

Map(center=[34.083568028567605, -118.61450081926833], controls=(WidgetControl(options=['position', 'transparen…

In [14]:
# Create the Normalised Burn Ratio Difference (dNBR)
dNBR = pre_med_nbr.subtract(post_med_nbr).rename("dNBR")

In [15]:
map_obj = geemap.Map()

# Create the centre point for the map
cnt_pt_x = roi_west + (roi_east - roi_west) / 2
cnt_pt_y = roi_south + (roi_north - roi_south) / 2
cnt_pt = ee.Geometry.Point([cnt_pt_x, cnt_pt_y])
# Specify the centre point and scale of the map
map_obj.centerObject(cnt_pt, 12)

# Add Google Satellite Imagery Basemap
gg_img_url = "http://www.google.cn/maps/vt?lyrs=s@189&gl=cn&x={x}&y={y}&z={z}"
map_obj.add_tile_layer(gg_img_url, name="Google Imagery", attribution="Google")

# Specify the visualisation options.
vis_params = {
    "min": [1320, 810, 20],
    "max": [4050, 4070, 1620],
    "bands": ["B8", "B11", "B4"],
}

# Add the Sentinel-2 Composite
map_obj.addLayer(pre_burn_s2_median, vis_params, "Pre-Burnt S2 Median")
map_obj.addLayer(post_burn_s2_median, vis_params, "Post-Burnt S2 Median")

# Specify the visualisation options.
vis_dnbr_params = {"min": [-0.2], "max": [0.2], "bands": ["dNBR"]}
map_obj.addLayer(dNBR, vis_dnbr_params, "dNBR")

map_obj

Map(center=[34.083568028567605, -118.61450081926833], controls=(WidgetControl(options=['position', 'transparen…

There is quite a lot of confusion with the water so before defining the burnt area we need to define a water mask - we'll use the pre-burn image for this.

In [16]:
# Calculate the Water Band Index (WBI) using the pre-fire median composite
pre_med_ndvi = pre_burn_s2_median.expression(
    "(NIR-RED)/(NIR+RED)",
    {
        "RED": pre_burn_s2_median.select("B4"),
        "NIR": pre_burn_s2_median.select("B8"),
    },
).rename("NDVI")

# Calculate the Water Band Index (WBI) using the pre-fire median composite
post_med_ndvi = post_burn_s2_median.expression(
    "(NIR-RED)/(NIR+RED)",
    {
        "RED": post_burn_s2_median.select("B4"),
        "NIR": post_burn_s2_median.select("B8"),
    },
).rename("NDVI")

# Calculate the Water Band Index (WBI) using the pre-fire median composite
pre_med_wbi = pre_burn_s2_median.expression(
    "(BLUE)/(NIR)",
    {
        "BLUE": pre_burn_s2_median.select("B2"),
        "NIR": pre_burn_s2_median.select("B8"),
    },
).rename("WBI")


In [17]:
# Threshold the dNBR to identify the burnt area as a unique region
burnt_area_tmp = dNBR.gt(0.1).rename("burnt_area").toInt()
burnt_area_tmp = burnt_area_tmp.mask(burnt_area_tmp.eq(1))

In [18]:
map_obj = geemap.Map()

# Create the centre point for the map
cnt_pt_x = roi_west + (roi_east - roi_west) / 2
cnt_pt_y = roi_south + (roi_north - roi_south) / 2
cnt_pt = ee.Geometry.Point([cnt_pt_x, cnt_pt_y])
# Specify the centre point and scale of the map
map_obj.centerObject(cnt_pt, 12)

# Add Google Satellite Imagery Basemap
gg_img_url = "http://www.google.cn/maps/vt?lyrs=s@189&gl=cn&x={x}&y={y}&z={z}"
map_obj.add_tile_layer(gg_img_url, name="Google Imagery", attribution="Google")

# Specify the visualisation options.
vis_params = {"min": [-0.2], "max": [0.2], "bands": ["NBR"]}
map_obj.addLayer(pre_med_nbr, vis_params, "Pre-Burnt NBR")
map_obj.addLayer(post_med_nbr, vis_params, "Post-Burnt NBR")

# Specify the visualisation options.
vis_dnbr_params = {"min": [-0.2], "max": [0.2], "bands": ["dNBR"]}
map_obj.addLayer(dNBR, vis_dnbr_params, "dNBR")

# Specify the visualisation options.
vis_wbi_params = {"min": [0], "max": [1], "bands": ["WBI"]}
map_obj.addLayer(pre_med_wbi, vis_wbi_params, "WBI")

# Specify the visualisation options.
vis_ndvi_params = {"min": [0], "max": [1], "bands": ["NDVI"]}
map_obj.addLayer(pre_med_ndvi, vis_ndvi_params, "NDVI (pre)")
map_obj.addLayer(post_med_ndvi, vis_ndvi_params, "NDVI (post)")

# Specify the visualisation options.
vis_burnt_params = {
    "min": 0,
    "max": 1,
    "palette": ["000000", "fcc200"],
    "bands": ["burnt_area"],
}
map_obj.addLayer(burnt_area_tmp, vis_burnt_params, "Burnt Area")

map_obj

Map(center=[34.083568028567605, -118.61450081926833], controls=(WidgetControl(options=['position', 'transparen…

In [19]:
# Threshold the WBI to identify the non-water areas as a unique region
water_area = pre_med_wbi.lt(0.6).rename("water").toInt()
water_area = water_area.mask(water_area.eq(1))

# Threshold the post-NDVI to identify the non-veg areas as a unique region
veg_post_msk_area = post_med_ndvi.lt(0.3).rename("veg").toInt()
veg_post_msk_area = veg_post_msk_area.mask(veg_post_msk_area.eq(1))


In [20]:
map_obj = geemap.Map()

# Create the centre point for the map
cnt_pt_x = roi_west + (roi_east - roi_west) / 2
cnt_pt_y = roi_south + (roi_north - roi_south) / 2
cnt_pt = ee.Geometry.Point([cnt_pt_x, cnt_pt_y])
# Specify the centre point and scale of the map
map_obj.centerObject(cnt_pt, 12)

# Add Google Satellite Imagery Basemap
gg_img_url = "http://www.google.cn/maps/vt?lyrs=s@189&gl=cn&x={x}&y={y}&z={z}"
map_obj.add_tile_layer(gg_img_url, name="Google Imagery", attribution="Google")

# Specify the visualisation options.
vis_water_params = {
    "min": 0,
    "max": 1,
    "palette": ["000000", "0000FF"],
    "bands": ["water"],
}
map_obj.addLayer(water_area, vis_water_params, "Water")

# Specify the visualisation options.
vis_veg_params = {
    "min": 0,
    "max": 1,
    "palette": ["000000", "00FF00"],
    "bands": ["veg"],
}
map_obj.addLayer(veg_post_msk_area, vis_veg_params, "Veg Msk")

map_obj

Map(center=[34.083568028567605, -118.61450081926833], controls=(WidgetControl(options=['position', 'transparen…

In [21]:
# Threshold the dNBR to identify the burnt area as a unique region
burnt_area_msk_img = dNBR.gt(0.1).rename("burnt_area").toInt()
burnt_area_msk_img = burnt_area_msk_img.mask(burnt_area_msk_img.eq(1)).updateMask(water_area.eq(1)).updateMask(veg_post_msk_area.eq(1))

In [22]:
map_obj = geemap.Map()

# Create the centre point for the map
cnt_pt_x = roi_west + (roi_east - roi_west) / 2
cnt_pt_y = roi_south + (roi_north - roi_south) / 2
cnt_pt = ee.Geometry.Point([cnt_pt_x, cnt_pt_y])
# Specify the centre point and scale of the map
map_obj.centerObject(cnt_pt, 12)

# Add Google Satellite Imagery Basemap
gg_img_url = "http://www.google.cn/maps/vt?lyrs=s@189&gl=cn&x={x}&y={y}&z={z}"
map_obj.add_tile_layer(gg_img_url, name="Google Imagery", attribution="Google")

vis_wbi_params = {"min": [0], "max": [1], "bands": ["WBI"]}
vis_ndvi_params = {"min": [0], "max": [1], "bands": ["NDVI"]}
map_obj.addLayer(pre_med_wbi, vis_wbi_params, "WBI")
map_obj.addLayer(post_med_ndvi, vis_ndvi_params, "NDVI (post)")

# Specify the visualisation options.
vis_dnbr_params = {"min": [-0.2], "max": [0.2], "bands": ["dNBR"]}

# Add the Landsat Composite
map_obj.addLayer(dNBR, vis_dnbr_params, "dNBR")

# Specify the visualisation options.
vis_burnt_params = {
    "min": 0,
    "max": 1,
    "palette": ["000000", "fcc200"],
    "bands": ["burnt_area"],
}

# Add the Landsat Composite
map_obj.addLayer(burnt_area_msk_img, vis_burnt_params, "Burnt Area")

map_obj

Map(center=[34.083568028567605, -118.61450081926833], controls=(WidgetControl(options=['position', 'transparen…

In [23]:
# Classify fire severity based on dNBR thresholds
fire_severity_img = (
    dNBR.expression(
        "dNBR < 0.1 ? 0 : "
        + "dNBR < 0.27 ? 1 : "
        + "dNBR < 0.44 ? 2 : "
        + "dNBR < 0.66 ? 3 : 4",
        {"dNBR": dNBR},
    )
    .rename("severity")
    .mask(burnt_area_msk_img.eq(1))
)

In [24]:
map_obj = geemap.Map()

# Create the centre point for the map
cnt_pt_x = roi_west + (roi_east - roi_west) / 2
cnt_pt_y = roi_south + (roi_north - roi_south) / 2
cnt_pt = ee.Geometry.Point([cnt_pt_x, cnt_pt_y])
# Specify the centre point and scale of the map
map_obj.centerObject(cnt_pt, 12)

# Add Google Maps Basemap
gg_maps_url = "https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}"
map_obj.add_tile_layer(gg_maps_url, name="Google Maps", attribution="Google")

# Add Google Satellite Imagery Basemap
gg_img_url = "http://www.google.cn/maps/vt?lyrs=s@189&gl=cn&x={x}&y={y}&z={z}"
map_obj.add_tile_layer(gg_img_url, name="Google Imagery", attribution="Google")

# Specify the visualisation options.
vis_burnt_params = {
    "min": 0,
    "max": 4,
    "palette": ["003f5c", "58508d", "bc5090", "ff6361", "ffa600"],
}

# Add the Landsat Composite
map_obj.addLayer(fire_severity_img, vis_burnt_params, "Burnt Severity")

map_obj

Map(center=[34.083568028567605, -118.61450081926833], controls=(WidgetControl(options=['position', 'transparen…

In [ ]:
# Create an image collection with the burnt area and fire severity images
out_img_collect = ee.ImageCollection([burnt_area_msk_img, fire_severity_img])
# Convert the image collection to a 2 band image which will be exported.
out_burn_img = out_img_collect.toBands().toInt().rename(["burnt_area", "severity"])

In [ ]:
out_burn_img

In [ ]:
# Output file name and directory
out_file_name = "la_2025_fire_extent_severity"
out_gdrive_dir = "burnt_area_outputs"

# Run export to save image to google drive
task = ee.batch.Export.image.toDrive(
    image=out_burn_img,
    description=out_file_name,
    folder=out_gdrive_dir,
    scale=30,
    region=tile_aoi,
    fileFormat="GeoTIFF",
    formatOptions={"cloudOptimized": True, "noData": out_no_data_val},
)
task.start()